## Rain prediction in Australia

#### Import required libraries

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE

import sklearn

from sklearn.svm import SVC
from sklearn.svm import SVR

from sklearn.impute import SimpleImputer

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler,OneHotEncoder, LabelEncoder

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score,roc_auc_score,precision_score, recall_score, f1_score,ConfusionMatrixDisplay,classification_report
from sklearn.metrics import mean_squared_error

import optuna

import xgboost as xgb
from xgboost import XGBClassifier

import joblib

import pipreqs

<br>
<br>
<br>
<br>
<br>

#### Read dataset

In [50]:
dataframe_clean_wo_outl_wo_corr = pd.read_csv(r"C:\Users\Lucio\Documents\Github\Next-day-rain-prediction\1- Data\2- Processed\dataframe_clean_wo_outl_wo_corr.csv", index_col=0)
dataframe_clean_wo_outl_wo_corr.head()

,Location,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,Humidity9am,Humidity3pm,Pressure9am,RainToday,RainTomorrow
0,Albury,0.6,W,44.0,W,WNW,71.0,22.0,1007.7,0.0,0.0
1,Albury,0.0,WNW,44.0,NNW,WSW,44.0,25.0,1010.6,0.0,0.0
2,Albury,0.0,WSW,46.0,W,WSW,38.0,30.0,1007.6,0.0,0.0
3,Albury,0.0,NE,24.0,SE,E,45.0,16.0,1017.6,0.0,0.0
4,Albury,1.0,W,41.0,ENE,NW,82.0,33.0,1010.8,0.0,0.0


dataframe_clean_wo_outl_wo_corr characteristics:
- Removed univariated outliers
- Removed variables with high collinearity

<br>
<br>
<br>
<br>
<br>

#### Encode Categorical Features

In [51]:
dataframe_encoded = pd.get_dummies(dataframe_clean_wo_outl_wo_corr)
dataframe_encoded.head()

,Rainfall,WindGustSpeed,Humidity9am,Humidity3pm,Pressure9am,RainToday,RainTomorrow,Location_Adelaide,Location_Albany,Location_Albury,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,0.6,44.0,71.0,22.0,1007.7,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0.0,44.0,44.0,25.0,1010.6,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0.0,46.0,38.0,30.0,1007.6,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0.0,24.0,45.0,16.0,1017.6,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1.0,41.0,82.0,33.0,1010.8,0.0,0.0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


<br>
<br>
<br>
<br>
<br>

## Model Tranining

#### Create X and y dataframes

In [52]:
X = dataframe_encoded[[c for c in dataframe_encoded if c != 'RainTomorrow']].values
y = dataframe_encoded[['RainTomorrow']]

In [53]:
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y,random_state=42, test_size=0.30)

#### Define optimization function

In [54]:
def objective_xgb(trial):
    param = {
        "verbosity": 1,
        "objective": "binary:logistic",
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        'eval_metric': 'error',
    }

    #param["booster"] == "gbtree"
    param["subsample"] = trial.suggest_float("subsample", 1e-8, 1.0, log=True)
    param["n_estimators"] = trial.suggest_int("n_estimators", 1, 1000)        
    param["max_depth"] = trial.suggest_int("max_depth", 1, 64)
    param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
    param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
    param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    
    ratio_majority_to_minority = len(y_train[y_train == 0]) / len(y_train[y_train == 1])  #Adjust weights based on rain/no-rain proportion
    
    bst = xgb.XGBClassifier(**param, scale_pos_weight=ratio_majority_to_minority)
    bst.fit(X_train, y_train)

    y_pred = bst.predict(X_val)
    accuracy = sklearn.metrics.accuracy_score(y_val, y_pred)

    return -accuracy  #Negative accuracy to maximize it (because 'eval_metric': 'error')

#### Applying StandardScaler

In [55]:
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X_sc = sc_X.fit_transform(X)
y_sc = sc_y.fit_transform(y)

In [56]:
X_sc_train, X_sc_val, y_sc_train, y_sc_val = sklearn.model_selection.train_test_split(X_sc, y_sc, random_state=42, test_size=0.30)

#### Applying SMOTE

In [57]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_sc_train, y_train)

#### Hyperparameter optimization with Optuna

In [58]:
study_xgb = optuna.create_study()
study_xgb.optimize(objective_xgb, n_trials=10)
study_xgb.best_params

[I 2023-11-22 23:21:40,448] A new study created in memory with name: no-name-415df3dd-ef5c-42cf-8ced-14cbc0b8257a
[I 2023-11-22 23:23:13,362] Trial 0 finished with value: -0.841257827014381 and parameters: {'booster': 'gbtree', 'lambda': 2.4378587270324184e-05, 'alpha': 0.00854914080541614, 'subsample': 0.0490288324443698, 'n_estimators': 203, 'max_depth': 61, 'eta': 5.937942797084505e-08, 'gamma': 0.18854338729868478, 'grow_policy': 'lossguide'}. Best is trial 0 with value: -0.841257827014381.
[W 2023-11-22 23:28:05,821] Trial 1 failed with parameters: {'booster': 'gbtree', 'lambda': 0.09154495097112088, 'alpha': 1.1291118862119675e-06, 'subsample': 0.3707341342795069, 'n_estimators': 980, 'max_depth': 14, 'eta': 0.6490805897592347, 'gamma': 0.014517542492053222, 'grow_policy': 'depthwise'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Lucio\miniconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    v

KeyboardInterrupt: 

In [59]:
xgb_params = study_xgb.best_params
xgb_params

{'booster': 'gbtree',
 'lambda': 2.4378587270324184e-05,
 'alpha': 0.00854914080541614,
 'subsample': 0.0490288324443698,
 'n_estimators': 203,
 'max_depth': 61,
 'eta': 5.937942797084505e-08,
 'gamma': 0.18854338729868478,
 'grow_policy': 'lossguide'}

#### Train model using best parameters
Adjust weights based on rain/no-rain proportion

In [60]:
ratio_majority_to_minority = len(y_sc_train[y_sc_train == 0]) / len(y_sc_train[y_sc_train == 1])  #Adjust weights based on rain/no-rain proportion

model = XGBClassifier(**xgb_params, silent=0, scale_pos_weight=ratio_majority_to_minority)

#Sin SMOTE
#model.fit(X_sc_train, y_sc_train, eval_set=[(X_sc_val, y_sc_val)], early_stopping_rounds=10, verbose=True)

#Con SMOTE
model.fit(X_train_resampled, y_train_resampled, eval_set=[(X_sc_val, y_val)], early_stopping_rounds=10, verbose=True)

C:\Users\Lucio\miniconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[23:28:15] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-logloss:0.69315
[1]	validation_0-logloss:0.69315
[2]	validation_0-logloss:0.69315
[3]	validation_0-logloss:0.69315
[4]	validation_0-logloss:0.69315
[5]	validation_0-logloss:0.69315
[6]	validation_0-logloss:0.69315
[7]	validation_0-logloss:0.69315
[8]	validation_0-logloss:0.69315
[9]	validation_0-logloss:0.69315
[10]	validation_0-logloss:0.69315
[11]	validation_0-logloss:0.69315
[12]	validation_0-logloss:0.69315
[13]	validation_0-logloss:0.69315
[14]	validation_0-logloss:0.69315
[15]	validation_0-logloss:0.69315
[16]	validation_0-logloss:0.69315
[17]	validation_0

KeyboardInterrupt: 

#### Predict using validation dataset

In [ ]:
y_predicted = model.predict(X_sc_val)
y_predicted

#### Model performance evaluation

In [ ]:
conf_matrix = confusion_matrix(y_val, y_predicted)

accuracy = accuracy_score(y_val, y_predicted)
precision = precision_score(y_val, y_predicted)
recall = recall_score(y_val, y_predicted)
f1 = f1_score(y_val, y_predicted)
roc_auc = roc_auc_score(y_val, y_predicted)

print("Confusion Matrix:\n", conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("ROC AUC:", roc_auc)

#### Save model

In [ ]:
ubi = r'C:\Users\Lucio\Documents\Github\Next-day-rain-prediction\3- Models/XGBClf_rain_pred.joblib'

joblib.dump(model, ubi)

Sin weights:
    Confusion Matrix:
 [[32302  1817]
 [ 4731  4749]]
Accuracy: 0.8498130691070896
Precision: 0.7232713981114834
Recall: 0.5009493670886076
F1-Score: 0.5919232207403714
ROC AUC: 0.7238472911822768

Con weights:
Confusion Matrix:
 [[27030  7089]
 [ 2289  7191]]
Accuracy: 0.7849033234707218
Precision: 0.5035714285714286
Recall: 0.7585443037974684
F1-Score: 0.6053030303030303
ROC AUC: 0.7753857542903635